In [2]:
import os
import urllib, cStringIO

import pymongo as pm

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
from __future__ import division
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64

from matplotlib.path import Path
import matplotlib.patches as patches

In [3]:
def strokes_to_lines(strokes):
    """
    Convert stroke-3 format to polyline format.
    List contains sublist of continuous line segments (strokes).    
    """
    x = 0
    y = 0
    lines = []
    line = []
    for i in range(len(strokes)):
        if strokes[i, 2] == 1:
            x += float(strokes[i, 0])
            y += float(strokes[i, 1])
            line.append([x, y])
            lines.append(line)
            line = []
        else:
            x += float(strokes[i, 0])
            y += float(strokes[i, 1])
            line.append([x, y])
    return lines

def polyline_pathmaker(lines):
    x = []
    y = []

    codes = [Path.MOVETO] # start with moveto command always
    for i,l in enumerate(lines):
        for _i,_l in enumerate(l):
            x.append(_l[0])
            y.append(_l[1])
            if _i<len(l)-1:
                codes.append(Path.LINETO) # keep pen on page
            else:
                if i != len(lines)-1: # final vertex
                    codes.append(Path.MOVETO)
    verts = zip(x,y)            
    return verts, codes

def path_renderer(verts, codes, save_path):
    fig = plt.figure(figsize=(6,6))    
    ax = plt.subplot(111)    
    padding = 10
    xlb = min([x for (x,y) in verts]) - padding
    xub = max([x for (x,y) in verts]) + padding
    ylb = min([y for (x,y) in verts]) - padding
    yub = max([y for (x,y) in verts]) + padding
    lb = min(xlb,ylb)
    ub = max(xub,yub)
    ax.axis('off')
    
    ax.set_xlim(lb,ub)
    ax.set_ylim(lb,ub)     
    
    if len(verts)>0:
        path = Path(verts, codes)
        patch = patches.PathPatch(path, facecolor='none', lw=5)
        ax.add_patch(patch)
        plt.gca().invert_yaxis() # y values increase as you go down in image
#         plt.show() 
    
    if len(save_path)>0:
        print save_path
        fig.savefig(save_path,bbox_inches='tight')
        # load, resize, then resave
        im = Image.open(save_path)
        im = im.resize((224,224), Image.ANTIALIAS)
        im.save(save_path)          

In [1]:
# path to npz files
path_npz = '/data2/jefan/quickDraw/npz'
path_png = '/data2/jefan/quickDraw/png'

# how many drawings from each category to include
sample_size = 500

In [53]:
# sorted list of all categories
paths = [os.path.join(path_npz,i) for i in sorted(os.listdir(path_npz))]

In [57]:
# loop through paths and render out
for path in paths:
    print path
    category = path.split('F')[1].split('.')[0]
    x = np.load(path)
    y = x['train']
    for i in np.arange(sample_size):    
        lines = strokes_to_lines(y[i])
        verts,codes = polyline_pathmaker(lines)
        save_path = os.path.join(path_png,category,'{:04d}.png'.format(i))   
        if not os.path.exists(os.path.join(path_png,category)):
            os.makedirs(os.path.join(path_png,category))
        path_renderer(verts, codes,save_path)
        plt.close('all')



### Jan 14 2018: render out smaller number of adult sketches from fewer categories (only the ones for which we have sufficient kid data

In [9]:
path_npz_mini = '/data2/jefan/quickDraw/npz'
path_png_mini = '/data2/jefan/quickDraw/png_mini'

# the categories for which we have enough kid data
sufficient_labels = ['banana',
                     'bird',
                     'boat',
                     'car',
                     'carrot',
                     'chair',
                     'couch',
                     'cup',
                     'flower',
                     'foot',
                     'frog',
                     'ice cream',
                     'phone',
                     'rabbit',
                     'shoe',
                     'train']

In [10]:
# sorted list of all categories
paths = [os.path.join(path_npz_mini,i) for i in sorted(os.listdir(path_npz_mini)) if i != 'ignore']

# how many drawings from each category to include
sample_size = 500

In [ ]:
# loop through paths and render out
for path in paths:
    print path
    category = path.split('F')[1].split('.')[0]
    x = np.load(path)
    y = x['train']
    for i in np.arange(sample_size):    
        lines = strokes_to_lines(y[i])
        verts,codes = polyline_pathmaker(lines)
        save_path = os.path.join(path_png_mini,category,'{:04d}.png'.format(i))   
        if not os.path.exists(os.path.join(path_png_mini,category)):
            os.makedirs(os.path.join(path_png_mini,category))
        path_renderer(verts, codes,save_path)
        plt.close('all')


/data2/jefan/quickDraw/npz/sketchrnn%2Fbanana.npz
/data2/jefan/quickDraw/png_mini/banana/0000.png
/data2/jefan/quickDraw/png_mini/banana/0001.png
/data2/jefan/quickDraw/png_mini/banana/0002.png
/data2/jefan/quickDraw/png_mini/banana/0003.png
/data2/jefan/quickDraw/png_mini/banana/0004.png
/data2/jefan/quickDraw/png_mini/banana/0005.png
/data2/jefan/quickDraw/png_mini/banana/0006.png
/data2/jefan/quickDraw/png_mini/banana/0007.png
/data2/jefan/quickDraw/png_mini/banana/0008.png
/data2/jefan/quickDraw/png_mini/banana/0009.png
/data2/jefan/quickDraw/png_mini/banana/0010.png
/data2/jefan/quickDraw/png_mini/banana/0011.png
/data2/jefan/quickDraw/png_mini/banana/0012.png
/data2/jefan/quickDraw/png_mini/banana/0013.png
/data2/jefan/quickDraw/png_mini/banana/0014.png
/data2/jefan/quickDraw/png_mini/banana/0015.png
/data2/jefan/quickDraw/png_mini/banana/0016.png
/data2/jefan/quickDraw/png_mini/banana/0017.png
/data2/jefan/quickDraw/png_mini/banana/0018.png
/data2/jefan/quickDraw/png_mini/banana

/data2/jefan/quickDraw/png_mini/banana/0172.png
/data2/jefan/quickDraw/png_mini/banana/0173.png
/data2/jefan/quickDraw/png_mini/banana/0174.png
/data2/jefan/quickDraw/png_mini/banana/0175.png
/data2/jefan/quickDraw/png_mini/banana/0176.png
/data2/jefan/quickDraw/png_mini/banana/0177.png
/data2/jefan/quickDraw/png_mini/banana/0178.png
/data2/jefan/quickDraw/png_mini/banana/0179.png
/data2/jefan/quickDraw/png_mini/banana/0180.png
/data2/jefan/quickDraw/png_mini/banana/0181.png
/data2/jefan/quickDraw/png_mini/banana/0182.png
/data2/jefan/quickDraw/png_mini/banana/0183.png
/data2/jefan/quickDraw/png_mini/banana/0184.png
/data2/jefan/quickDraw/png_mini/banana/0185.png
/data2/jefan/quickDraw/png_mini/banana/0186.png
/data2/jefan/quickDraw/png_mini/banana/0187.png
/data2/jefan/quickDraw/png_mini/banana/0188.png
/data2/jefan/quickDraw/png_mini/banana/0189.png
/data2/jefan/quickDraw/png_mini/banana/0190.png
/data2/jefan/quickDraw/png_mini/banana/0191.png
/data2/jefan/quickDraw/png_mini/banana/0

/data2/jefan/quickDraw/png_mini/banana/0343.png
/data2/jefan/quickDraw/png_mini/banana/0344.png
/data2/jefan/quickDraw/png_mini/banana/0345.png
/data2/jefan/quickDraw/png_mini/banana/0346.png
/data2/jefan/quickDraw/png_mini/banana/0347.png
/data2/jefan/quickDraw/png_mini/banana/0348.png
/data2/jefan/quickDraw/png_mini/banana/0349.png
/data2/jefan/quickDraw/png_mini/banana/0350.png
/data2/jefan/quickDraw/png_mini/banana/0351.png
/data2/jefan/quickDraw/png_mini/banana/0352.png
/data2/jefan/quickDraw/png_mini/banana/0353.png
/data2/jefan/quickDraw/png_mini/banana/0354.png
/data2/jefan/quickDraw/png_mini/banana/0355.png
/data2/jefan/quickDraw/png_mini/banana/0356.png
/data2/jefan/quickDraw/png_mini/banana/0357.png
/data2/jefan/quickDraw/png_mini/banana/0358.png
/data2/jefan/quickDraw/png_mini/banana/0359.png
/data2/jefan/quickDraw/png_mini/banana/0360.png
/data2/jefan/quickDraw/png_mini/banana/0361.png
/data2/jefan/quickDraw/png_mini/banana/0362.png
/data2/jefan/quickDraw/png_mini/banana/0

/data2/jefan/quickDraw/png_mini/bird/0015.png
/data2/jefan/quickDraw/png_mini/bird/0016.png
/data2/jefan/quickDraw/png_mini/bird/0017.png
/data2/jefan/quickDraw/png_mini/bird/0018.png
/data2/jefan/quickDraw/png_mini/bird/0019.png
/data2/jefan/quickDraw/png_mini/bird/0020.png
/data2/jefan/quickDraw/png_mini/bird/0021.png
/data2/jefan/quickDraw/png_mini/bird/0022.png
/data2/jefan/quickDraw/png_mini/bird/0023.png
/data2/jefan/quickDraw/png_mini/bird/0024.png
/data2/jefan/quickDraw/png_mini/bird/0025.png
/data2/jefan/quickDraw/png_mini/bird/0026.png
/data2/jefan/quickDraw/png_mini/bird/0027.png
/data2/jefan/quickDraw/png_mini/bird/0028.png
/data2/jefan/quickDraw/png_mini/bird/0029.png
/data2/jefan/quickDraw/png_mini/bird/0030.png
/data2/jefan/quickDraw/png_mini/bird/0031.png
/data2/jefan/quickDraw/png_mini/bird/0032.png
/data2/jefan/quickDraw/png_mini/bird/0033.png
/data2/jefan/quickDraw/png_mini/bird/0034.png
/data2/jefan/quickDraw/png_mini/bird/0035.png
/data2/jefan/quickDraw/png_mini/bi

/data2/jefan/quickDraw/png_mini/bird/0196.png
/data2/jefan/quickDraw/png_mini/bird/0197.png
/data2/jefan/quickDraw/png_mini/bird/0198.png
/data2/jefan/quickDraw/png_mini/bird/0199.png
/data2/jefan/quickDraw/png_mini/bird/0200.png
/data2/jefan/quickDraw/png_mini/bird/0201.png
/data2/jefan/quickDraw/png_mini/bird/0202.png
/data2/jefan/quickDraw/png_mini/bird/0203.png
/data2/jefan/quickDraw/png_mini/bird/0204.png
/data2/jefan/quickDraw/png_mini/bird/0205.png
/data2/jefan/quickDraw/png_mini/bird/0206.png
/data2/jefan/quickDraw/png_mini/bird/0207.png
/data2/jefan/quickDraw/png_mini/bird/0208.png
/data2/jefan/quickDraw/png_mini/bird/0209.png
/data2/jefan/quickDraw/png_mini/bird/0210.png
/data2/jefan/quickDraw/png_mini/bird/0211.png
/data2/jefan/quickDraw/png_mini/bird/0212.png
/data2/jefan/quickDraw/png_mini/bird/0213.png
/data2/jefan/quickDraw/png_mini/bird/0214.png
/data2/jefan/quickDraw/png_mini/bird/0215.png
/data2/jefan/quickDraw/png_mini/bird/0216.png
/data2/jefan/quickDraw/png_mini/bi

...

/data2/jefan/quickDraw/png_mini/rabbit/0179.png
/data2/jefan/quickDraw/png_mini/rabbit/0180.png
/data2/jefan/quickDraw/png_mini/rabbit/0181.png
/data2/jefan/quickDraw/png_mini/rabbit/0182.png
/data2/jefan/quickDraw/png_mini/rabbit/0183.png
/data2/jefan/quickDraw/png_mini/rabbit/0184.png
/data2/jefan/quickDraw/png_mini/rabbit/0185.png
/data2/jefan/quickDraw/png_mini/rabbit/0186.png
/data2/jefan/quickDraw/png_mini/rabbit/0187.png
/data2/jefan/quickDraw/png_mini/rabbit/0188.png
/data2/jefan/quickDraw/png_mini/rabbit/0189.png
/data2/jefan/quickDraw/png_mini/rabbit/0190.png
/data2/jefan/quickDraw/png_mini/rabbit/0191.png
/data2/jefan/quickDraw/png_mini/rabbit/0192.png
/data2/jefan/quickDraw/png_mini/rabbit/0193.png
/data2/jefan/quickDraw/png_mini/rabbit/0194.png
/data2/jefan/quickDraw/png_mini/rabbit/0195.png
/data2/jefan/quickDraw/png_mini/rabbit/0196.png
/data2/jefan/quickDraw/png_mini/rabbit/0197.png
/data2/jefan/quickDraw/png_mini/rabbit/0198.png
/data2/jefan/quickDraw/png_mini/rabbit/0

/data2/jefan/quickDraw/png_mini/rabbit/0351.png
/data2/jefan/quickDraw/png_mini/rabbit/0352.png
/data2/jefan/quickDraw/png_mini/rabbit/0353.png
/data2/jefan/quickDraw/png_mini/rabbit/0354.png
/data2/jefan/quickDraw/png_mini/rabbit/0355.png
/data2/jefan/quickDraw/png_mini/rabbit/0356.png
/data2/jefan/quickDraw/png_mini/rabbit/0357.png
/data2/jefan/quickDraw/png_mini/rabbit/0358.png
/data2/jefan/quickDraw/png_mini/rabbit/0359.png
/data2/jefan/quickDraw/png_mini/rabbit/0360.png
/data2/jefan/quickDraw/png_mini/rabbit/0361.png
/data2/jefan/quickDraw/png_mini/rabbit/0362.png
/data2/jefan/quickDraw/png_mini/rabbit/0363.png
/data2/jefan/quickDraw/png_mini/rabbit/0364.png
/data2/jefan/quickDraw/png_mini/rabbit/0365.png
/data2/jefan/quickDraw/png_mini/rabbit/0366.png
/data2/jefan/quickDraw/png_mini/rabbit/0367.png
/data2/jefan/quickDraw/png_mini/rabbit/0368.png
/data2/jefan/quickDraw/png_mini/rabbit/0369.png
/data2/jefan/quickDraw/png_mini/rabbit/0370.png
/data2/jefan/quickDraw/png_mini/rabbit/0

/data2/jefan/quickDraw/png_mini/shoe/0023.png
/data2/jefan/quickDraw/png_mini/shoe/0024.png
/data2/jefan/quickDraw/png_mini/shoe/0025.png
/data2/jefan/quickDraw/png_mini/shoe/0026.png
/data2/jefan/quickDraw/png_mini/shoe/0027.png
/data2/jefan/quickDraw/png_mini/shoe/0028.png
/data2/jefan/quickDraw/png_mini/shoe/0029.png
/data2/jefan/quickDraw/png_mini/shoe/0030.png
/data2/jefan/quickDraw/png_mini/shoe/0031.png
/data2/jefan/quickDraw/png_mini/shoe/0032.png
/data2/jefan/quickDraw/png_mini/shoe/0033.png
/data2/jefan/quickDraw/png_mini/shoe/0034.png
/data2/jefan/quickDraw/png_mini/shoe/0035.png
/data2/jefan/quickDraw/png_mini/shoe/0036.png
/data2/jefan/quickDraw/png_mini/shoe/0037.png
/data2/jefan/quickDraw/png_mini/shoe/0038.png
/data2/jefan/quickDraw/png_mini/shoe/0039.png
/data2/jefan/quickDraw/png_mini/shoe/0040.png
/data2/jefan/quickDraw/png_mini/shoe/0041.png
/data2/jefan/quickDraw/png_mini/shoe/0042.png
/data2/jefan/quickDraw/png_mini/shoe/0043.png
/data2/jefan/quickDraw/png_mini/sh

/data2/jefan/quickDraw/png_mini/shoe/0202.png
/data2/jefan/quickDraw/png_mini/shoe/0203.png
/data2/jefan/quickDraw/png_mini/shoe/0204.png
/data2/jefan/quickDraw/png_mini/shoe/0205.png
/data2/jefan/quickDraw/png_mini/shoe/0206.png
/data2/jefan/quickDraw/png_mini/shoe/0207.png
/data2/jefan/quickDraw/png_mini/shoe/0208.png
/data2/jefan/quickDraw/png_mini/shoe/0209.png
/data2/jefan/quickDraw/png_mini/shoe/0210.png
/data2/jefan/quickDraw/png_mini/shoe/0211.png
/data2/jefan/quickDraw/png_mini/shoe/0212.png
/data2/jefan/quickDraw/png_mini/shoe/0213.png
/data2/jefan/quickDraw/png_mini/shoe/0214.png
/data2/jefan/quickDraw/png_mini/shoe/0215.png
/data2/jefan/quickDraw/png_mini/shoe/0216.png
/data2/jefan/quickDraw/png_mini/shoe/0217.png
/data2/jefan/quickDraw/png_mini/shoe/0218.png
/data2/jefan/quickDraw/png_mini/shoe/0219.png
/data2/jefan/quickDraw/png_mini/shoe/0220.png
/data2/jefan/quickDraw/png_mini/shoe/0221.png
/data2/jefan/quickDraw/png_mini/shoe/0222.png
/data2/jefan/quickDraw/png_mini/sh

/data2/jefan/quickDraw/png_mini/shoe/0382.png
/data2/jefan/quickDraw/png_mini/shoe/0383.png
/data2/jefan/quickDraw/png_mini/shoe/0384.png
/data2/jefan/quickDraw/png_mini/shoe/0385.png
/data2/jefan/quickDraw/png_mini/shoe/0386.png
/data2/jefan/quickDraw/png_mini/shoe/0387.png
/data2/jefan/quickDraw/png_mini/shoe/0388.png
/data2/jefan/quickDraw/png_mini/shoe/0389.png
/data2/jefan/quickDraw/png_mini/shoe/0390.png
/data2/jefan/quickDraw/png_mini/shoe/0391.png
/data2/jefan/quickDraw/png_mini/shoe/0392.png
/data2/jefan/quickDraw/png_mini/shoe/0393.png
/data2/jefan/quickDraw/png_mini/shoe/0394.png
/data2/jefan/quickDraw/png_mini/shoe/0395.png
/data2/jefan/quickDraw/png_mini/shoe/0396.png
/data2/jefan/quickDraw/png_mini/shoe/0397.png
/data2/jefan/quickDraw/png_mini/shoe/0398.png
/data2/jefan/quickDraw/png_mini/shoe/0399.png
/data2/jefan/quickDraw/png_mini/shoe/0400.png
/data2/jefan/quickDraw/png_mini/shoe/0401.png
/data2/jefan/quickDraw/png_mini/shoe/0402.png
/data2/jefan/quickDraw/png_mini/sh